# Generate video dataset with motion

This notebook provides code to generate a synthetic video dataset with various direction of a sprite movement in them. We can use this dataset as input to a CNN.

In [1]:
import os
from PIL import Image, ImageDraw
from random import randint
import subprocess as sp
import math

global FFMPEG_BIN
FFMPEG_BIN = "c:\\vsprojects\\ffmpeg\\ffmpeg.exe"

EAST = 0
SOUTH_EAST = 1
SOUTH = 2
SOUTH_WEST = 3
WEST = 4
NORTH_WEST = 5
NORTH = 6
NORTH_EAST = 7

NA_DIR = -1

# This represents the amount of pixels the sprite moves in each frame
MOTION_DELTA = 3

FRAME_WIDTH = 300
FRAME_HEIGHT = 300

SPRITE_WIDTH = 25
SPRITE_HEIGHT = 25

### Helper function to save video frames

This is a helper function that simply saves a frame as an image at the provided file location

In [2]:
def saveimg(filename, img):
    if not os.path.isfile(filename):
        #print("Saving", filename )
        img.save(filename)
    else:
        print("File already exists", filename)

### Get direction of motion

Get the direction of the sprite movement based on the slope of the line of motion

In [3]:
def getDirection(slope):
    
    direction = "NA"
    
    if ((slope < math.tan(math.pi/8)) and (slope > math.tan(-math.pi/8))):
        direction = EAST
    elif ((slope > math.tan(math.pi/8)) and (slope < math.tan(3*math.pi/8))):
        direction = SOUTH_EAST   
    elif ((slope < math.tan(-math.pi/8)) and (slope > math.tan(-math.pi/4))):
        direction = NORTH_EAST
    elif ((slope < math.tan(-math.pi/4)) and (slope > math.tan(-3*math.pi/8))):
        direction = SOUTH_WEST
    elif ((slope > math.tan(3*math.pi/8)) or (slope < math.tan(5*math.pi/8))):
        direction = SOUTH
        
    return direction

In [4]:
def getMirrorDirection(direction):
    
    if (direction == EAST):
        direction = WEST
    elif (direction == SOUTH):
        direction = NORTH
    elif (direction == NORTH_EAST):
        direction = SOUTH_WEST
    elif (direction == SOUTH_WEST):
        direction = NORTH_EAST
    elif (direction == SOUTH_EAST):
        direction = NORTH_WEST
    else:
        direction = NA_DIR
        
    return direction


### Render the sprite

The functions below render the sprite in a video frame and save the frame as an image. The sprite is a circle in this case but the code below can be modified to render a sprite of a different shape.

In [5]:
def render_ew(dirname, frame, index, frame_width, frame_height, sprite_width, sprite_height, slope, intercept, 
              color, fill_color, frame_type):        
    
    x1 = frame
    x2 = x1 + sprite_width

    y1 = slope * x1 + intercept
    y2 = y1 + sprite_height

    img = Image.new(frame_type, (frame_width, frame_height), color)    
    draw = ImageDraw.Draw(img)        
    draw.ellipse((x1, y1, x2, y2), fill = fill_color)

    saveimg(dirname + str(index) + ".png", img)     

In [6]:
def render_ns(dirname, frame, index, frame_width, frame_height, sprite_width, sprite_height, slope, intercept, 
              color, fill_color, frame_type):        
    
    y1 = frame
    y2 = y1 + sprite_height

    x1 = (y1 - intercept) / slope 
    x2 = x1 + sprite_width

    img = Image.new(frame_type, (frame_width, frame_height), color)    
    draw = ImageDraw.Draw(img)        
    draw.ellipse((x1, y1, x2, y2), fill = fill_color)

    saveimg(dirname + str(index) + ".png", img)   

In [7]:
def render_east(dirname, frame_width, frame_height, sprite_height, sprite_width, slope, intercept, 
                color, fill_color, frame_type):        
    
    index = 0
    
    for frame in range(0, frame_width, MOTION_DELTA):
        render_ew(dirname, frame, index, frame_width, frame_height, sprite_width, sprite_height, slope, intercept, 
                  color, fill_color, frame_type)      
        index = index + 1

In [8]:
def render_west(dirname, frame_width, frame_height, sprite_width, sprite_height, slope, intercept, 
                color, fill_color, frame_type):
            
    index = 0
    
    for frame in range(frame_width - sprite_width, 0, -MOTION_DELTA):
        render_ew(dirname, frame, index, frame_width, frame_height, sprite_width, sprite_height, slope, intercept, 
                  color, fill_color, frame_type)      
        index = index + 1        

In [9]:
def render_south(dirname, frame_width, frame_height, sprite_width, sprite_height, slope, intercept, 
                 color, fill_color, frame_type):
            
    index = 0
    
    for frame in range(0, frame_height, MOTION_DELTA):
        render_ns(dirname, frame, index, frame_width, frame_height, sprite_width, sprite_height, slope, intercept, 
                  color, fill_color, frame_type)      
        index = index + 1   

In [10]:
def render_north(dirname, frame_width, frame_height, sprite_width, sprite_height, slope, intercept, 
                 color, fill_color, frame_type):
            
    index = 0
    
    for frame in range(frame_height - sprite_height, 0, -MOTION_DELTA):
        render_ns(dirname, frame, index, frame_width, frame_height, sprite_width, sprite_height, slope, intercept, 
                  color, fill_color, frame_type)      
        index = index + 1  

### Helper function to generate a black and white image

The function below generates a black and white image. It randomly selects the background color to be black or white and then selects the other color as the shape fill color.

In [11]:
def genColorsForBWImage(flip = True):
    
    backColor = 0
    
    if (flip):
        backColor = randint(0, 1)        
    
    fillColor = 0

    if (backColor == 0):
        fillColor = 255
    else:
        backColor = 255
        
    return backColor, fillColor

### Helper function to generate a grayscale image

The function below generates a grayscale image. It randomly selects the background color and the color of the shape

In [12]:
def genColorsForGrayscaleImage():
    
    backColor = randint(0, 255)        
    fillColor = randint(0, 255)
    
    return backColor, fillColor

### Helper function to generate a RGB image

This function generated a RGB image by generating a random background and shape fill color

In [13]:
def genColorsForRGBImage():
            
    r = randint(0, 255)
    g = randint(0, 255)
    b = randint(0, 255)        

    fr = randint(0, 255)
    fg = randint(0, 255)
    fb = randint(0, 255)      

    backColor = (r, g, b)
    fillColor = (fr, fg, fb)

    return backColor, fillColor

### Helper function to generate video frames

The video frames are generated with a random color for the background and the sprite and also a random direction of motion for the sprite

In [18]:
def genFrames(dirname, frame_width, frame_height, sprite_width, sprite_height, frameType):
    if not os.path.exists(dirname):
        os.makedirs(dirname)
           
    sprite_height = randint(10, sprite_height)
    sprite_width = sprite_height 

    if ((frameType == "BW") or (frameType == "GS")):        
        frame_type = 'L'
    else:
        frame_type = 'RGB'
        
    backColor = 0
    fillColor = 0       

    if (frameType == "RGB"):
        backColor, fillColor = genColorsForRGBImage()
    elif (frameType == "BW"):
        backColor, fillColor = genColorsForBWImage(False)
    elif (frameType == "GS"):
        backColor, fillColor = genColorsForGrayscaleImage()                    
    
    d = randint(0, 1)
    
    if (d == 0):
        x1 = 0
        y1 = randint(0, frame_height - sprite_height) 
        x2 = sprite_width
        y2 = y1 + sprite_height 

        endpoint_x = frame_width - sprite_width
        endpoint_y = randint(0, frame_height - sprite_height)
    else:
        y1 = 0
        x1 = randint(0, frame_width - sprite_width)
        y2 = sprite_height
        x2 = x1 + sprite_height
        
        endpoint_y = frame_height - sprite_height
        endpoint_x = randint(0, frame_width - sprite_width)
        
        if (x1 == endpoint_x):
            endpoint_x = endpoint_x + 1
        
        
    #Calculate slope and intercept for the sprite motion 
    slope = (y1 - endpoint_y)/(x1 - endpoint_x)    
    intercept = y1 - x1 * slope
    
    #print(slope)
        
    direction = getDirection(slope)                 
    
    m = randint(0, 1)
    
    if (d == 0):
        if (m == 0):
            render_east(dirname, frame_width, frame_height, sprite_width, sprite_height, slope, intercept, 
                        backColor, fillColor, frame_type)
        else:
            render_west(dirname, frame_width, frame_height, sprite_width, sprite_height, slope, intercept, 
                        backColor, fillColor, frame_type)
            direction = getMirrorDirection(direction)
    else:
        if (m == 0):
            render_south(dirname, frame_width, frame_height, sprite_width, sprite_height, slope, intercept, 
                         backColor, fillColor, frame_type)
        else:
            render_north(dirname, frame_width, frame_height, sprite_width, sprite_height, slope, intercept, 
                         backColor, fillColor, frame_type)
            
            direction = getMirrorDirection(direction)
            
    #print (direction)
    return direction

### Generate the video file

Use ffmpeg to generate the video file from the set of images 

In [19]:
def genVideo(dirname, videoFileName, fps, frame_width, frame_height, sprite_width, sprite_height, frameType):
    global FFMPEG_BIN
    
    if not os.path.isfile(videoFileName):
        direction = genFrames(dirname, frame_width, frame_height, sprite_width, sprite_height, frameType)
        command = [ FFMPEG_BIN,
            '-r', str(fps),
            '-i', dirname + '%d.png',
            '-c:v', 'libx264',
            videoFileName ]
        
        
        try:
            sp.check_call(command)
        except sp.CalledProcessError as e:
            print("ERROR for ffmpeg: {reason}".format(reason=e))
            exit(-1)
        except OSError as e:
            print("OSERROR for ffmpeg: {reason}".format(reason=e))
            exit(-1)                
        
        if os.path.isfile(videoFileName):
            filelist = [ f for f in os.listdir(dirname) if f.endswith(".png") ]
            for f in filelist:                
                os.remove(os.path.join(dirname, f))
        else:
            print('Video was not generated')
    else:
        print('Video ' + videoFileName + ' already exists')
        
    return direction

### Generate list of videos

Generate a list of videos with motion in different directions

In [20]:
def genVideoList(temp_dirname, video_dirname, index_filename, fps, frame_width, frame_height, 
                 sprite_width, sprite_height, frameType, numVideos, start_idx, print_frequency = 1000):
    
    f = open(index_filename, "a+")
    
    for i in range(start_idx, numVideos + start_idx):
        if ((i % print_frequency) == 0):
            print("Generating video " + str(i))
        
        videoFileName = video_dirname + str(i) + ".mp4"
        direction = genVideo(temp_dirname, videoFileName, fps, frame_width, frame_height, sprite_width, sprite_height, 
                             frameType)
        
        f.write(videoFileName + "\t" + str(direction) + "\n")
        
    print("Generating video list done")
    
    f.close()    

In [21]:
genVideoList("./temp/", "./train-mono-motion-dataset/", "./train-mono-motion.txt", 30, FRAME_WIDTH, FRAME_HEIGHT, 
             SPRITE_WIDTH, SPRITE_HEIGHT, "BW", 40000, 0)
genVideoList("./temp/", "./test-mono-motion-dataset/", "./test-mono-motion.txt", 30, FRAME_WIDTH, FRAME_HEIGHT, 
             SPRITE_WIDTH, SPRITE_HEIGHT, "BW", 8000, 0)

Generating video 0
Generating video 1000
Generating video 2000
Generating video 3000
Generating video 4000
Generating video 5000
Generating video 6000
Generating video 7000
Generating video 8000
Generating video 9000
Generating video 10000
Generating video 11000
Generating video 12000
Generating video 13000
Generating video 14000
Generating video 15000
Generating video 16000
Generating video 17000
Generating video 18000
Generating video 19000
Generating video 20000
Generating video 21000
Generating video 22000
Generating video 23000
Generating video 24000
Generating video 25000
Generating video 26000
Generating video 27000
Generating video 28000
Generating video 29000
Generating video 30000
Generating video 31000
Generating video 32000
Generating video 33000
Generating video 34000
Generating video 35000
Generating video 36000
Generating video 37000
Generating video 38000
Generating video 39000
Generating video list done
Generating video 0
Generating video 1000
Generating video 2000
Gen

In [ ]:
genVideoList("./temp/", "./train-rgb-motion-dataset/", "./train-rgb-motion.txt", 30, 600, 600, 
             50, 50, "RGB", 1, 0)